In [1]:
!jupyter nbconvert --to=python DetCalc_profile.ipynb

[NbConvertApp] Converting notebook Alk_bias_profile.ipynb to python
[NbConvertApp] Writing 2378 bytes to Alk_bias_profile.py


In [2]:
class DetCalc_profile:   

    def __init__(self,runname,resultpath,savepath,mesh,
                 first_year,last_year,savefig=False, regional=False):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.fyear = first_year
        self.lyear = last_year
        self.savefig = savefig
        self.regional = regional
        
        import matplotlib.pyplot as plt
        import numpy as np
        from netCDF4 import Dataset
        from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        import pyfesom2 as pf
        
#         from load_interp_GLODAP_python3 import GLODAPdata
        
        years = np.arange(first_year, last_year+1,1)
        #mesh       = pf.load_mesh(self.meshpath)
        meshdiag = pf.get_meshdiag(mesh)
        runid      =  self.runname
        
        # get depth of export production
        i_ep_depth = pf.ind_for_depth(100,mesh)
        nod_depth = meshdiag.zbar_n_bottom
        ep_depth = nod_depth[i_ep_depth]
        
        unitsDetCalc = 'DetCalc Flux [[Pg C y$^{-1}$]'
        
        
        # load data -------------------------------------------------------------------------------------

#         Alkfesom = pf.get_data(resultpath, "Alk", years, mesh,how="mean", compute=True, runid=runid, silent=True)
        DetCalc = pf.get_data(resultpath, "DetCalc", years, mesh, how=None, compute=False, runid=self.runname, silent=True)
        DetCalc = DetCalc.resample(time='YS').mean(dim='time')
        
        Vdet = 0.0288 * 100. + 20. ## sinking velocity
        DetCalc = 365. * DetCalc * 12.01 * Vdet /1e18 # [mmol/m3] => [mg/m2/yr] => [Pg C/yr]
         
        detct=DetCalc[:,:,i_ep_depth]
        EP_calc_timeseries = pf.areasum_data(detct,mesh,mask)    
        
        # Load and derive profiles

        nod_area = np.ma.masked_equal(meshdiag.nod_area.values, 0)
        mask = pf.get_mask(mesh, "Global Ocean")

        Alkfesom_by_area = ((np.ma.masked_equal(Alkfesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
        Alkglodap_by_area = ((np.ma.masked_equal(Alkglodap[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

        Alkfesom_weighted_Global = Alkfesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
        Alkglodap_weighted_Global = Alkglodap_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
        
        ######
        Bias = Alkfesom_weighted_Global-Alkglodap_weighted_Global
        ######
        self.Bias_out = Bias